# 10. k-최근접 이웃(kNN)과 나이브 베이즈

## 학습 목표
- kNN의 거리 기반 분류 원리 이해
- 거리 메트릭 (Euclidean, Manhattan, Minkowski) 학습
- 최적 k값 선택 방법 습득
- 나이브 베이즈의 확률 기반 분류 이해
- Gaussian, Multinomial, Bernoulli NB 비교
- 텍스트 분류 적용

In [ ]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.datasets import (
    load_iris, load_breast_cancer, load_diabetes, load_digits,
    make_classification, fetch_20newsgroups
)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.spatial.distance import euclidean, cityblock, minkowski, chebyshev
from time import time

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)

## 1. k-최근접 이웃 (kNN) 개념

kNN은 게으른 학습(Lazy Learning) 알고리즘입니다.

**동작 원리**:
1. 새로운 데이터가 들어오면
2. 학습 데이터에서 가장 가까운 k개의 이웃을 찾음
3. k개 이웃의 다수결(분류) 또는 평균(회귀)으로 예측

**특징**:
- 학습 시 모델 생성 없음 (모든 데이터 저장)
- 비모수적 방법 (데이터 분포 가정 불필요)
- 예측 시간이 느림

In [ ]:
# 2D 데이터로 kNN 시각화
X, y = make_classification(
    n_samples=100, n_features=2, n_redundant=0,
    n_informative=2, n_clusters_per_class=1, random_state=42
)

# 여러 k값 비교
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
k_values = [1, 5, 15]

for ax, k in zip(axes, k_values):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X, y)

    # 결정 경계
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))

    Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    ax.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='black')
    ax.set_title(f'k = {k}\nAccuracy = {knn.score(X, y):.3f}')
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

## 2. kNN 기본 사용법

In [ ]:
# 데이터 로드
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42
)

# kNN 분류기
knn = KNeighborsClassifier(
    n_neighbors=5,           # k값
    weights='uniform',       # 가중치: 'uniform' 또는 'distance'
    algorithm='auto',        # 알고리즘: 'auto', 'ball_tree', 'kd_tree', 'brute'
    metric='minkowski',      # 거리 측정: 'euclidean', 'manhattan', 'minkowski'
    p=2                      # minkowski p값 (2=euclidean, 1=manhattan)
)

knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print("kNN 분류 결과:")
print(f"  정확도: {accuracy_score(y_test, y_pred):.4f}")
print("\n분류 리포트:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

## 3. 거리 측정 방법

kNN의 핵심은 거리 계산입니다.

주요 거리 메트릭:
- **유클리드 (Euclidean, L2)**: d = √Σ(xi - yi)²
- **맨해튼 (Manhattan, L1)**: d = Σ|xi - yi|
- **민코프스키 (Minkowski)**: d = (Σ|xi - yi|^p)^(1/p)
- **체비셰프 (Chebyshev, L∞)**: d = max(|xi - yi|)

In [ ]:
# 거리 측정 예시
point1 = np.array([1, 2, 3])
point2 = np.array([4, 5, 6])

print("거리 측정 예시:")
print(f"  Point 1: {point1}")
print(f"  Point 2: {point2}")
print()
print(f"  유클리드 거리:     {euclidean(point1, point2):.4f}")
print(f"  맨해튼 거리:       {cityblock(point1, point2):.4f}")
print(f"  민코프스키 (p=3):  {minkowski(point1, point2, p=3):.4f}")
print(f"  체비셰프 거리:     {chebyshev(point1, point2):.4f}")

In [ ]:
# 거리 메트릭별 성능 비교
metrics = ['euclidean', 'manhattan', 'chebyshev']

print("거리 메트릭별 성능 (Iris):")
print("-" * 40)
for metric in metrics:
    knn = KNeighborsClassifier(n_neighbors=5, metric=metric)
    knn.fit(X_train, y_train)
    acc = knn.score(X_test, y_test)
    print(f"  {metric:12s}: {acc:.4f}")

## 4. 최적 k값 선택

k값이 너무 작으면 과적합, 너무 크면 과소적합이 발생합니다.
교차 검증으로 최적 k를 찾습니다.

In [ ]:
# k값에 따른 성능 변화
k_range = range(1, 31)
train_scores = []
test_scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    train_scores.append(knn.score(X_train, y_train))
    test_scores.append(knn.score(X_test, y_test))

# 시각화
plt.figure(figsize=(10, 6))
plt.plot(k_range, train_scores, 'o-', label='Train')
plt.plot(k_range, test_scores, 's-', label='Test')
plt.xlabel('k (Number of Neighbors)')
plt.ylabel('Accuracy')
plt.title('kNN: k vs Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(k_range[::2])
plt.tight_layout()
plt.show()

# 최적 k 찾기
best_k = k_range[np.argmax(test_scores)]
print(f"최적 k: {best_k}")
print(f"최고 테스트 정확도: {max(test_scores):.4f}")

In [ ]:
# 교차 검증으로 k 선택
k_range = range(1, 31)
cv_scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())

# 시각화
plt.figure(figsize=(10, 6))
plt.plot(k_range, cv_scores, 'o-', color='green')
plt.xlabel('k')
plt.ylabel('Cross-Validation Accuracy')
plt.title('kNN: k Selection with 5-Fold Cross-Validation')
plt.grid(True, alpha=0.3)
plt.xticks(k_range[::2])
plt.tight_layout()
plt.show()

best_k_cv = k_range[np.argmax(cv_scores)]
print(f"교차 검증 최적 k: {best_k_cv}")
print(f"최고 CV 정확도: {max(cv_scores):.4f}")

## 5. 가중 kNN (Weighted kNN)

거리에 따라 이웃의 가중치를 조절합니다.

- **uniform**: 모든 이웃에 동일한 가중치
- **distance**: 가까운 이웃에 더 큰 가중치 (weight = 1/distance)

In [ ]:
# 가중치 방식 비교
weights = ['uniform', 'distance']

print("가중치 방식 비교:")
print("-" * 40)
for weight in weights:
    knn = KNeighborsClassifier(n_neighbors=5, weights=weight)
    knn.fit(X_train, y_train)
    acc = knn.score(X_test, y_test)
    print(f"  {weight:10s}: {acc:.4f}")

In [ ]:
# 거리 가중 kNN 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, weight in zip(axes, weights):
    knn = KNeighborsClassifier(n_neighbors=15, weights=weight)
    knn.fit(X[:, :2], y)

    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))

    Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    ax.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='black')
    ax.set_title(f'weights = {weight}')
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

## 6. kNN 회귀

kNN은 회귀 문제에도 사용할 수 있습니다.
k개 이웃의 평균으로 예측합니다.

In [ ]:
# 데이터 로드
diabetes = load_diabetes()
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(
    diabetes.data, diabetes.target, test_size=0.2, random_state=42
)

# 스케일링 (kNN은 거리 기반이므로 필수)
scaler = StandardScaler()
X_train_d_scaled = scaler.fit_transform(X_train_d)
X_test_d_scaled = scaler.transform(X_test_d)

# kNN 회귀
knn_reg = KNeighborsRegressor(n_neighbors=5, weights='distance')
knn_reg.fit(X_train_d_scaled, y_train_d)
y_pred_d = knn_reg.predict(X_test_d_scaled)

print("kNN 회귀 결과:")
print(f"  MSE: {mean_squared_error(y_test_d, y_pred_d):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test_d, y_pred_d)):.4f}")
print(f"  R²: {r2_score(y_test_d, y_pred_d):.4f}")

## 7. kNN 알고리즘 비교

대용량 데이터에서는 탐색 알고리즘 선택이 중요합니다.

- **brute**: 전수 탐색 (O(n))
- **kd_tree**: KD-Tree 사용 (저차원에 효율적)
- **ball_tree**: Ball-Tree 사용 (고차원에 효율적)

In [ ]:
# 알고리즘별 시간 비교
algorithms = ['brute', 'kd_tree', 'ball_tree']

print("알고리즘별 시간 비교:")
print("-" * 60)
for algo in algorithms:
    knn = KNeighborsClassifier(n_neighbors=5, algorithm=algo)

    # 학습 시간
    start = time()
    knn.fit(X_train, y_train)
    fit_time = time() - start

    # 예측 시간
    start = time()
    knn.predict(X_test)
    pred_time = time() - start

    print(f"  {algo:10s}: fit={fit_time:.4f}s, predict={pred_time:.4f}s")

## 8. 나이브 베이즈 (Naive Bayes)

### 베이즈 정리

**P(y|X) = P(X|y) × P(y) / P(X)**

- P(y|X): 사후 확률 (특성이 주어졌을 때 클래스 확률)
- P(X|y): 우도 (클래스가 주어졌을 때 특성 확률)
- P(y): 사전 확률 (클래스의 기본 확률)
- P(X): 증거 (특성의 확률)

### 나이브 가정

모든 특성이 서로 독립적이라고 가정:
**P(X|y) = P(x₁|y) × P(x₂|y) × ... × P(xₙ|y)**

## 9. 가우시안 나이브 베이즈

연속형 특성이 가우시안(정규) 분포를 따른다고 가정합니다.
**P(xi|y) = N(xi; μy, σy)**

In [ ]:
# 가우시안 나이브 베이즈
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_nb = gnb.predict(X_test)

print("가우시안 나이브 베이즈 결과:")
print(f"  정확도: {accuracy_score(y_test, y_pred_nb):.4f}")

# 학습된 파라미터 확인
print(f"\n클래스 사전 확률: {gnb.class_prior_}")
print(f"\n클래스별 평균 (처음 2개 특성):")
print(gnb.theta_[:, :2])
print(f"\n클래스별 분산 (처음 2개 특성):")
print(gnb.var_[:, :2])

In [ ]:
# 확률 예측
y_proba = gnb.predict_proba(X_test[:5])

print("확률 예측 (처음 5개):")
print(f"클래스: {iris.target_names}")
print(y_proba)
print(f"\n예측 클래스: {gnb.predict(X_test[:5])}")
print(f"실제 클래스: {y_test[:5]}")

## 10. 다항 나이브 베이즈 - 텍스트 분류

이산형/카운트 특성에 사용하며, 주로 텍스트 분류(단어 빈도)에 활용됩니다.

**P(xi|y) = (Nyi + α) / (Ny + αn)**

- α: Laplace smoothing 파라미터 (Zero frequency 문제 해결)

In [ ]:
# 뉴스 데이터 로드
categories = ['sci.space', 'rec.sport.baseball', 'talk.politics.misc']
newsgroups = fetch_20newsgroups(
    subset='train',
    categories=categories,
    remove=('headers', 'footers', 'quotes'),
    random_state=42
)

print(f"뉴스 데이터: {len(newsgroups.data)} 기사")
print(f"카테고리: {categories}")
print(f"\n첫 번째 기사 (일부):\n{newsgroups.data[0][:200]}...")

In [ ]:
# 텍스트 벡터화
vectorizer = CountVectorizer(max_features=5000, stop_words='english')
X_news = vectorizer.fit_transform(newsgroups.data)
y_news = newsgroups.target

print(f"벡터 크기: {X_news.shape}")
print(f"특성 수: {len(vectorizer.get_feature_names_out())}")

# 학습/테스트 분할
X_train_news, X_test_news, y_train_news, y_test_news = train_test_split(
    X_news, y_news, test_size=0.2, random_state=42
)

In [ ]:
# 다항 나이브 베이즈
mnb = MultinomialNB(alpha=1.0)  # alpha: Laplace smoothing
mnb.fit(X_train_news, y_train_news)

y_pred_news = mnb.predict(X_test_news)

print("다항 나이브 베이즈 (텍스트 분류) 결과:")
print(f"  정확도: {mnb.score(X_test_news, y_test_news):.4f}")
print("\n분류 리포트:")
print(classification_report(y_test_news, y_pred_news, target_names=categories))

In [ ]:
# 각 클래스의 가장 중요한 단어
feature_names = vectorizer.get_feature_names_out()

print("각 클래스별 상위 10개 단어:")
print("=" * 60)
for i, category in enumerate(categories):
    top_indices = mnb.feature_log_prob_[i].argsort()[-10:][::-1]
    top_words = [feature_names[idx] for idx in top_indices]
    print(f"\n{category}:")
    print(f"  {', '.join(top_words)}")

## 11. 베르누이 나이브 베이즈

이진 특성(0/1)에 사용하며, 단어의 존재 여부로 텍스트를 분류합니다.

In [ ]:
# 이진 벡터화 (단어 존재 여부만)
binary_vectorizer = CountVectorizer(max_features=5000, binary=True, stop_words='english')
X_binary = binary_vectorizer.fit_transform(newsgroups.data)

X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(
    X_binary, y_news, test_size=0.2, random_state=42
)

# 베르누이 나이브 베이즈
bnb = BernoulliNB(alpha=1.0)
bnb.fit(X_train_bin, y_train_bin)

print("베르누이 나이브 베이즈 결과:")
print(f"  정확도: {bnb.score(X_test_bin, y_test_bin):.4f}")

## 12. 나이브 베이즈 모델 비교

In [ ]:
# 숫자 이미지 데이터
digits = load_digits()
X_train_dig, X_test_dig, y_train_dig, y_test_dig = train_test_split(
    digits.data, digits.target, test_size=0.2, random_state=42
)

# 세 가지 나이브 베이즈 비교
models = {
    'Gaussian NB': GaussianNB(),
    'Multinomial NB': MultinomialNB(),
    'Bernoulli NB': BernoulliNB()
}

print("나이브 베이즈 모델 비교 (Digits):")
print("-" * 50)
for name, model in models.items():
    model.fit(X_train_dig, y_train_dig)
    acc = model.score(X_test_dig, y_test_dig)
    print(f"  {name:18s}: {acc:.4f}")

## 13. 온라인 학습 (Incremental Learning)

나이브 베이즈는 `partial_fit`으로 온라인 학습이 가능합니다.
대용량 데이터나 스트리밍 데이터에 유용합니다.

In [ ]:
# 온라인 학습 시뮬레이션
gnb_online = GaussianNB()

# 배치 학습
batch_size = 50
n_batches = len(X_train) // batch_size

for i in range(n_batches):
    start = i * batch_size
    end = start + batch_size
    X_batch = X_train[start:end]
    y_batch = y_train[start:end]

    # 첫 배치에서 클래스 정의
    if i == 0:
        gnb_online.partial_fit(X_batch, y_batch, classes=np.unique(y_train))
    else:
        gnb_online.partial_fit(X_batch, y_batch)

print("온라인 학습 결과:")
print(f"  배치 수: {n_batches}")
print(f"  배치 크기: {batch_size}")
print(f"  정확도: {gnb_online.score(X_test, y_test):.4f}")

## 14. kNN vs 나이브 베이즈 비교

In [ ]:
# 유방암 데이터
cancer = load_breast_cancer()
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    cancer.data, cancer.target, test_size=0.2, random_state=42
)

# 스케일링
scaler = StandardScaler()
X_train_c_scaled = scaler.fit_transform(X_train_c)
X_test_c_scaled = scaler.transform(X_test_c)

# 모델 비교
models = {
    'kNN (k=5)': KNeighborsClassifier(n_neighbors=5),
    'kNN (weighted)': KNeighborsClassifier(n_neighbors=5, weights='distance'),
    'Gaussian NB': GaussianNB()
}

print("kNN vs 나이브 베이즈 비교 (Breast Cancer):")
print("=" * 50)

for name, model in models.items():
    if 'kNN' in name:
        model.fit(X_train_c_scaled, y_train_c)
        acc = model.score(X_test_c_scaled, y_test_c)
    else:
        model.fit(X_train_c, y_train_c)
        acc = model.score(X_test_c, y_test_c)
    print(f"  {name:18s}: {acc:.4f}")

## 15. 간단한 텍스트 분류 예제

In [ ]:
# 간단한 감성 분류
texts = [
    "I love this movie", "Great film", "Excellent acting",
    "Amazing performance", "Wonderful story",
    "Terrible movie", "Bad film", "Worst movie ever",
    "Horrible acting", "Disappointing story"
]
labels = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]  # 1: positive, 0: negative

# TF-IDF 벡터화
tfidf = TfidfVectorizer()
X_sentiment = tfidf.fit_transform(texts)

# 나이브 베이즈 학습
mnb_sentiment = MultinomialNB()
mnb_sentiment.fit(X_sentiment, labels)

# 새로운 텍스트 분류
new_texts = [
    "This is a great movie",
    "I hate this film",
    "Excellent performance and story",
    "Terrible and disappointing"
]
X_new = tfidf.transform(new_texts)
predictions = mnb_sentiment.predict(X_new)
probabilities = mnb_sentiment.predict_proba(X_new)

print("감성 분류 결과:")
print("=" * 60)
for text, pred, prob in zip(new_texts, predictions, probabilities):
    sentiment = "Positive" if pred == 1 else "Negative"
    confidence = max(prob) * 100
    print(f"'{text}'")
    print(f"  → {sentiment} (신뢰도: {confidence:.1f}%)\n")

## 정리

### kNN 요약

| 파라미터 | 설명 | 권장 |
|----------|------|------|
| **n_neighbors** | 이웃 수 (k) | 교차 검증으로 선택 |
| **weights** | 가중치 방식 | 'distance' 추천 |
| **metric** | 거리 측정 | 'euclidean' 기본 |
| **algorithm** | 탐색 알고리즘 | 'auto' |

**특징**:
- 게으른 학습 (학습 시간 없음)
- 예측 시간 느림 (O(n·d))
- 스케일링 필수
- 고차원에서 성능 저하 (차원의 저주)

### 나이브 베이즈 요약

| 종류 | 특성 타입 | 주요 용도 |
|------|-----------|----------|
| **GaussianNB** | 연속형 (정규 분포) | 일반 분류 |
| **MultinomialNB** | 카운트/빈도 | 텍스트 분류 |
| **BernoulliNB** | 이진 (0/1) | 단어 존재 여부 |

**특징**:
- 매우 빠름 (학습 O(n·d), 예측 O(d))
- 적은 데이터로도 잘 작동
- 고차원 데이터에 효과적
- 온라인 학습 가능
- 특성 독립성 가정 (현실에서 위반 가능)

### kNN vs 나이브 베이즈

| 특성 | kNN | 나이브 베이즈 |
|------|-----|---------------|
| **학습 시간** | O(1) | O(n·d) |
| **예측 시간** | O(n·d) | O(d) |
| **메모리** | 높음 | 낮음 |
| **스케일링** | 필수 | 불필요 |
| **고차원** | 약함 | 강함 |
| **해석성** | 직관적 | 확률 기반 |

### 다음 단계
- Clustering (K-Means, DBSCAN)
- Dimensionality Reduction (PCA, t-SNE)
- Ensemble methods (Stacking, Voting)